In [1]:
require(tidyverse)
require(rstan)
require(ggplot2)
require(mvtnorm)
rstan_options(auto_write = TRUE)
require(bayesplot)
missing_val=-100

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: rstan

Loading required package: StanHeaders

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)


Attaching package: ‘rstan’


The following object is masked from ‘package:tidyr’:

    extract


Loading required package: mvtnorm

Loading required package: bayesplot

This is bayesplot version 1.7.1

- Online documentation and v

In [2]:
path = file.path('/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/analysis', '/20200305_2_stage_impute_beta_missing')
dir.create(path)
setwd(path)

Warning message in dir.create(path):
“'/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/analysis//20200305_2_stage_impute_beta_missing' already exists”


In [3]:
df_nmr = read_csv('/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/11_20_2019_even_odds_interaction_prior_1/nmr_metabolites_scaled_matrix.csv')
metabolites = colnames(df_nmr)[1:27]

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.



In [4]:
#TARGET_MISSING_RATE=0.3
get_alpha_beta_for_target_rate = function(df,metabolites,TARGET_MISSING_RATE) {
    missing_rate = df_nmr[,metabolites] %>% summarise_all(function(x) { mean(x == min(x))  })
    missing_rate_numeric = missing_rate %>% t() %>% as.numeric
    mu = mean( missing_rate_numeric )
    sigma_2 = var(missing_rate_numeric)
    RATE = TARGET_MISSING_RATE/mu
    mu = RATE * mu
    sigma_2 = RATE * sigma_2
    mu
    sigma_2
    nu = mu * (1 - mu) / sigma_2 - 1
    alpha = mu * nu
    beta = (1 - mu) * nu
    list(alpha=alpha,beta=beta,mu=mu,sigma_2=sigma_2)
}

get_alpha_beta_for_target_rate(df_nmr,metabolites,0.3)

$alpha
[1] 2.435085

$beta
[1] 5.681866

$mu
[1] 0.3

$sigma_2
[1] 0.02303402

In [5]:
died = df_nmr$died_90_day
sigma_0 = cov(df_nmr[died == 0,c(1:27)])
mu_0 = colMeans(df_nmr[died == 0,c(1:27)]) 
sigma_1 = cov(df_nmr[died == 1,c(1:27)])
mu_1 = colMeans(df_nmr[died == 1,c(1:27)]) 

### Generation code

In [6]:
make_df = function(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.5,censor=TRUE,max_missing=0.6,alpha=-1,beta=-1) {
    diffs = abs( mu_1 - mu_0) / sqrt( (diag(sigma_0)^2)/n_0 + (diag(sigma_1)^2)/n_1  )
    #print(diffs)
    m = length(diffs)
    diffs = diffs/sum(diffs)
    #print(diffs)
    ind = seq(1,m)
    set_non_zero = sample(ind,size=ceiling(frac_sig * m),prob=diffs)
    set_zero = setdiff(ind,set_non_zero)
    truth = rep(FALSE,m)
    truth[set_non_zero] = TRUE
    mu_0[set_zero] = 0
    mu_1[set_zero] = 0
    X_0 = rmvnorm(n = n_0, mean = mu_0, sigma = sigma_0) 
    X_1 = rmvnorm(n = n_1, mean = mu_1, sigma = sigma_1)
    df_sample = data.frame(rbind(X_0,X_1))
    df_sample$died_90_day = c( rep(0,n_0),rep(1,n_1)  )
    
    df_censored = df_sample
    df_naive_impute = df_sample
    n_var = length(mu_0)
    thresholds = rep(0,n_var)
    est.thresholds = rep(0,n_var)
    est.naive_impute = rep(0,n_var)
    missing_rates=rep(0,n_var)
    if(censor == TRUE) {
        
        missing_rates = rbeta(n_var,shape1 = alpha, shape2 = beta)
        # only allow a missing rate up to a certain level
        missing_rates = sapply(missing_rates, function(x) { min(max_missing,x) })
        
        #missing_rates = rep(MISSING_RATE, n_var)
        
        for(i in 1:n_var) {
            thresholds[i] = quantile( df_censored[, i], missing_rates[i] )
            df_censored[ (df_censored[, i] < thresholds[i]), i ] = NA
            est.thresholds[i] = min( df_censored[ , i ], na.rm = T)
            est.naive_impute[i] = log( min( exp(df_censored[ , i ]), na.rm = T) / 2 )
            df_naive_impute[ (df_naive_impute[, i] < thresholds[i]), i ] = est.thresholds[i]
        }
    }
    
    
    list(df_sample=df_sample,truth=truth, df_censored=df_censored, df_naive_impute=df_naive_impute, missing_rates=missing_rates, thresholds=thresholds, est.thresholds=est.thresholds, est.naive_impute=est.naive_impute)
}

### Stage 1 Imputation model

In [7]:
stage_1_impute = "
functions {
    real normal_ub_rng(real mu, real sigma, real ub) {
      real p_ub = normal_cdf(ub, mu, sigma);
      real u = uniform_rng(0, p_ub);
      real y = mu + sigma * inv_Phi(u);
      return y;
    }
}
data {
    int<lower=0> N;
    int<lower=0> N_var;
    int<lower=0> N_miss;
    int<lower=0> N_use_impute;

    matrix[ N, N_var ] x_naive_impute;
    matrix[ N, N_var ] x_is_missing;

    //contains the indicies of x_naive_imp to use for imputation
    int x_use_impute[N_use_impute, N_var ];

    //Threshold for imputation
    //Separate threshold per metabolite
    vector[N_var] threshold;
    

    // This vector contains the missing row index, column index
    int x_missing[ N_miss, 2 ];
    
}
parameters {

    //betas for each imputed value
    matrix[ N_use_impute, N_var ] beta_impute_raw;
    vector[ N_var ] alpha_impute;
    vector<lower=0>[ N_var ] sigma_impute;
    real<lower=0> sigma_beta_impute_x;

}
transformed parameters {
    matrix[ N_use_impute, N_var ] beta_impute;
    for(j in 1:N_var) {
        beta_impute[,j] = beta_impute_raw[,j] * sigma_beta_impute_x;
    }
}
model {
    sigma_beta_impute_x ~ std_normal();
    // FORGOT TO SET PRIOR ON alpha_x_pred and sigma_impute

    for(j in 1:N_var) {
        real y[N];
        vector[N_use_impute] beta_x_pred;
        int x_use_ind[N_use_impute];
        matrix[N, N_use_impute] x_pred_use;
        real alpha_x_pred;
        real sigma_x_pred;
        //predicted target
        vector[N] xj_pred;
        //target
        vector[N] xj;
        int observed[N];
        
        //ADD TO LIKELIHOOD
        beta_impute_raw[, j] ~ std_normal();
    
        //ASSIGN VALS
        xj = x_naive_impute[, j];
        //dont make a prediction using itself
        //this should be copying the values according to ref
        beta_x_pred = beta_impute[, j];
        alpha_x_pred = alpha_impute[j];
        x_use_ind = x_use_impute[, j];
        sigma_x_pred = sigma_impute[j];
        x_pred_use = x_naive_impute[, x_use_ind];
        
        //Make pred
        xj_pred = x_pred_use * beta_x_pred + alpha_x_pred;
        
        
        //observed = (x_is_missing[, j] == 0);
        
        // target +=  normal_lpdf(xj | xj_pred, sigma_x_pred);

        for(i in 1:N) {
            real mu_x_pred;
            mu_x_pred = xj_pred[i];
            if( x_is_missing[i,j] == 0) {
                // if observed use pdf for observed value
                target +=  normal_lpdf(xj[i] | mu_x_pred, sigma_x_pred);
            } else {
                // if censored, use normal CDF
                target +=  normal_lcdf(threshold[j] | mu_x_pred, sigma_x_pred);
            }
        }
    }
}
generated quantities {
    real x_impute_mean[ N_miss ];
    real x_impute[ N_miss ];

    for(i in 1:N_miss) {
        //These correspond to the rows and columns in the original data frame
        int metab_row;
        int metab_col;
        row_vector[N_use_impute] x_pred_use;
        vector[N_use_impute] beta_x_pred;
        real alpha_x_pred;
        int x_use_ind[N_use_impute];
        real mu_x;
        real sd_x;
        real ub_x;

        metab_row = x_missing[i,1];
        metab_col = x_missing[i,2];
        x_use_ind = x_use_impute[, metab_col ];
        beta_x_pred = beta_impute[, metab_col ];
        alpha_x_pred = alpha_impute[ metab_col ];
        x_pred_use = x_naive_impute[metab_row, x_use_ind];

        mu_x = x_pred_use * beta_x_pred + alpha_x_pred;
        sd_x =  sigma_impute[ metab_col ];
        ub_x = threshold[ metab_col ];
        x_impute_mean[ i ] = mu_x;
        x_impute[i] = normal_ub_rng(mu_x, sd_x, ub_x);
    }
}

"

In [8]:
stage_1_impute_code_file = 'stage_1_impute.stan'
write_lines(x = stage_1_impute, path = stage_1_impute_code_file)
stage_1_impute_model = stan_model(file = stage_1_impute_code_file, verbose = FALSE)

In [9]:
stage_1_impute_get_init_vals = function(data) {
    n_var = data$N_var
    N_use_impute = data$N_use_impute
    # matrix[ N_use_impute, N_var ] beta_impute_raw;
    beta_impute_raw = matrix( rep(0, data$N_use_impute * n_var), ncol=n_var)
    alpha_impute = rep(0,n_var)
    x_use_impute = data$x_use_impute
    sigma_impute = rep(1, n_var)
    
    for(i in 1:n_var) {
        x_imp_vals_use = data$x_naive_impute[, x_use_impute[,i]   ]
        y_out_imp = data$x_naive_impute[,i]
        m_out = glm(y_out_imp ~ x_imp_vals_use,family="gaussian")
        coefs_imp = summary(m_out)$coef
        alpha_impute[i] = coefs_imp[1,1]
        for(k in 1:N_use_impute) {
            beta_impute_raw[k,i] = coefs_imp[1 + k,1]
        }
        sigma_impute[i] = sum(m_out$residuals^2) / m_out$df.residual
    }
    
    sigma_beta_impute_x = sqrt( sum( beta_impute_raw^2 ) / (prod(dim(beta_impute_raw)) - 1) ) 
    
    inits = list(
        sigma_impute = sigma_impute,
        sigma_beta_impute_x = sigma_beta_impute_x,
        alpha_impute = alpha_impute,
        beta_impute_raw = beta_impute_raw / sigma_beta_impute_x
    )
    
    inits
    
}

In [13]:
stage_1_impute_make_data = function(x_censored, y, threshold=NULL,
                                         N_use_impute=8) {
    
    if(is.null(threshold)) {
        threshold = apply(x_censored,MARGIN=2,FUN=function(x) { min(x,na.rm = T)})
    }
    
    naive_impute_val = log( exp(threshold) / 2)
    
    n = dim(x_censored)[1]
    N_var = dim(x_censored)[2]
    N_miss = sum(is.na(x_censored))
    
    x_naive_impute = x_censored %>% select_all()
    x_is_missing = apply( is.na(x_censored), FUN=as.numeric, MARGIN=c(1,2))             
    for(j in 1:N_var) {
        x_naive_impute[is.na(x_naive_impute[,j]),j] = naive_impute_val[j]
    }            
    # create a matrix that contains the N_use_impute indicies of the most correlated
    # variables with the metabolite of interest
    # each column contains a metabolite corresponding with x_censored
    # each row corresponds to the index of the metabolite most correlated with it up to N_use_impute
    # rows
    var_cors = cor(x_naive_impute,use = "complete")
    diag(var_cors) = 0
    get_inds_to_use_per_metabolite = function(x) { which( rank(-abs(x)) <= N_use_impute)   }
    x_use_impute = apply(var_cors,MARGIN=1,FUN=get_inds_to_use_per_metabolite )

    x_censored[is.na(x_censored)] = missing_val

    x_censored = as.matrix(x_censored)
    x_missing = matrix(nrow = N_miss, ncol = 2)
    x_missing_init = rep(0,N_miss)
    x_missing_init_raw = rep(0,N_miss)
    
    # make a table were the first column is the row number in x_censored
    # second column contains the metabolite column number in x_censored
    col1 = c()
    col2 = c()
    for(j in 1:dim(x_censored)[2]) {
        missing_in_col = which(x_censored[,j] == missing_val)
        col1 = c(col1, missing_in_col)
        col2 = c(col2, rep(j,length(missing_in_col)))
    }
    x_missing[,1] = col1
    x_missing[,2] = col2
    
    if( N_miss > 0 ) {
        # make a vector with naive imputation values
        for(i in 1:dim(x_missing)[1]) {
            row = x_missing[i,1]
            col = x_missing[i,2]
            x_missing_init[i] = x_naive_impute[row,col]
            x_missing_init_raw[i] = x_missing_init[i] - threshold[col]
        }
    }

    data = list(
            N = n,
            N_miss = N_miss,
            N_var = N_var,
            x_raw = x_censored,
            x_naive_impute = as.matrix(x_naive_impute),
            N_use_impute = N_use_impute,
            x_use_impute = x_use_impute,
            x_missing = x_missing,
            x_missing_init = x_missing_init,
            x_missing_init_raw = x_missing_init_raw,
            x_is_missing = x_is_missing,
            threshold = threshold,
            y=y
        )

    #str(data)
    #mean(x_censored == missing_val)
    #data$N_miss / (data$N * data$N_var)

    data   
}


In [14]:
replicate_init = function(init_data,chains=4) {
    init = list()
    for(i in 1:chains){
        init[paste0(i)] = list(init_data)
    }
    init
} 

In [16]:
set.seed(0)
n_0=100
n_1=100
beta_params = get_alpha_beta_for_target_rate(df_nmr,metabolites,TARGET_MISSING_RATE = 0.3)
#out = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.4,censor=TRUE,
#                  max_missing = 0.4, alpha = beta_params$alpha, beta=beta_params$beta)

out = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.4,censor=FALSE,
                  max_missing = 0.4, alpha = beta_params$alpha, beta=beta_params$beta)


write_rds(out,'debug_df.rds')

x_censored = out$df_censored[,metabolites]
y=out$df_censored$died_90_day
sum(is.na(x_censored))

stage_1_data = stage_1_impute_make_data(x_censored, y, N_use_impute=8)
stage_1_init_data = replicate_init(
        stage_1_impute_get_init_vals(stage_1_data), chains=1
    )
str(stage_1_data)

[1] 0

List of 13
 $ N                 : int 200
 $ N_miss            : int 0
 $ N_var             : int 27
 $ x_raw             : num [1:200, 1:27] -0.2539 0.1963 0.0995 1.0577 -0.5308 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_naive_impute    : num [1:200, 1:27] -0.2539 0.1963 0.0995 1.0577 -0.5308 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ N_use_impute      : num 8
 $ x_use_impute      : int [1:8, 1:27] 3 15 17 18 19 22 26 27 1 4 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_missing         : int[0 , 1:2] 
 $ x_missing_init    : num(0) 
 $ x_missing_init_raw: num(0) 
 $ x_is_missing      : num [1:200, 1:27] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..

In [17]:
stage_1_fit = sampling(stage_1_impute_model,  data = stage_1_data, init=stage_1_init_data, cores=1, chains=1, iter = 2000, control=list(adapt_delta=0.8))



SAMPLING FOR MODEL 'stage_1_impute' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 87.36 seconds (Warm-up)
Chain 1:                57.19 seconds (Sampling)
Chain 1:                144.55 se

In [18]:
df_sum = data.frame(summary(stage_1_fit)$summary) 
df_sum$PARAM = rownames(summary(stage_1_fit)$summary)
df_sum %>% arrange(n_eff) %>% head(10)
df_sum %>% arrange(-abs(Rhat)) %>% head(10)

,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,-5512.6520737,1.1956927349,16.06832048,-5.543797e+03,-5.523679e+03,-5513.1013050,-5501.2417105,-5480.7377332,180.5934,0.9995295,lp__
2,0.1972392,0.0006506298,0.01056270,1.780939e-01,1.895429e-01,0.1964747,0.2043881,0.2188030,263.5613,0.9991405,sigma_beta_impute_x
3,3.6329327,0.0134496092,0.27568563,3.097374e+00,3.445525e+00,3.6250536,3.8110687,4.1945714,420.1544,0.9991785,"beta_impute_raw[7,16]"
4,4.3375261,0.0148969168,0.31698955,3.720797e+00,4.117323e+00,4.3296303,4.5447819,4.9611346,452.7903,0.9990645,"beta_impute_raw[7,25]"
5,3.3370878,0.0127093304,0.28440287,2.791659e+00,3.141564e+00,3.3299810,3.5263381,3.9184802,500.7519,0.9997764,"beta_impute_raw[8,17]"
6,3.4715000,0.0103898173,0.31538750,2.843631e+00,3.245802e+00,3.4727668,3.6934118,4.0503065,921.4529,0.9993288,"beta_impute_raw[5,27]"
7,1.4982265,0.0069840063,0.22234685,1.061910e+00,1.343338e+00,1.5059335,1.6490110,1.9177824,1013.5676,0.9995534,"beta_impute_raw[4,4]"
8,0.5832399,0.0092835255,0.30322833,2.510396e-02,3.715368e-01,0.5748025,0.7738567,1.2013517,1066.8754,0.9990317,"beta_impute_raw[6,4]"
9,0.1147143,0.0017639371,0.05902539,5.266292e-03,7.296193e-02,0.1127369,0.1549204,0.2334397,1119.7256,0.9990989,"beta_impute[6,4]"


,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.51640867,0.011842587,0.41118668,-0.30030119,0.25400458,0.48908695,0.787742967,1.36391972,1205.549,1.003561,"beta_impute_raw[3,21]"
2,0.23161804,0.001574323,0.07256845,0.08728896,0.18406437,0.23234613,0.278254824,0.37151915,2124.750,1.003547,"beta_impute[1,9]"
3,0.10170174,0.002306241,0.08079381,-0.05972333,0.04961109,0.09612933,0.154734287,0.26551895,1227.289,1.003540,"beta_impute[3,21]"
4,-0.22552211,0.009534655,0.38590639,-0.98936137,-0.48054038,-0.23625986,0.036739931,0.49465571,1638.151,1.003409,"beta_impute_raw[6,21]"
5,-0.04452559,0.001859412,0.07585322,-0.19545827,-0.09451529,-0.04647340,0.007485014,0.09729545,1664.167,1.003375,"beta_impute[6,21]"
6,1.17621741,0.008417771,0.36959853,0.44055025,0.92927941,1.17041269,1.418843823,1.93140420,1927.819,1.002851,"beta_impute_raw[1,9]"
7,-0.11382482,0.006487158,0.28030502,-0.68483057,-0.29978539,-0.10539689,0.056003849,0.42525222,1867.037,1.002350,"beta_impute_raw[4,16]"
8,-0.02235305,0.001294869,0.05498725,-0.13486865,-0.05828562,-0.02075392,0.011042828,0.08294994,1803.319,1.002322,"beta_impute[4,16]"
9,0.79403343,0.008869983,0.38462335,0.06309948,0.53065739,0.77645881,1.070965202,1.53488170,1880.294,1.002314,"beta_impute_raw[5,24]"


### Stage 2 regression code

In [19]:
stage_2_regress = "
data {
    int<lower=0> N;
    int<lower=0> N_var;
    int<lower=0> N_miss;

    matrix[ N, N_var ] x_raw;

    int<lower=0,upper=1> y[N];

    //Threshold for imputation
    //Separate threshold per metabolite
    vector[N_var] threshold;
    
    // This vector contains the missing row index, column index
    int x_missing[ N_miss, 2 ];
    real x_missing_mean[ N_miss ];
    real x_missing_sd[ N_miss ];
    
}
parameters {
    // Needed for imputation
    vector<upper=0>[ N_miss ] x_impute_raw;
    
    //regression on y per metabolite in x_raw columns
    vector[N_var] beta_x;
    vector[N_var] alpha_x;

    real<lower=0> sigma_beta_x;
    real<lower=1> nu_x;

}
transformed parameters {
    // https://mc-stan.org/docs/2_18/stan-users-guide/vectors-with-varying-bounds.html
    vector<upper=0>[ N_miss ] x_impute;
    x_impute = x_impute_raw;
    for(i in 1:N_miss) {
        int metab_row = x_missing[i,1];
        int metab_col = x_missing[i,2];
        x_impute[i] = x_impute[i] + threshold[metab_col];
    }

}
model {
    matrix[N, N_var] x_merge;
    //copy matrix
    x_merge = x_raw;
    
    //fill in the values we are going to impute
    for(i in 1:N_miss) {
        int metab_row = x_missing[i,1];
        int metab_col = x_missing[i,2];
        x_merge[metab_row,metab_col] = x_impute[ i ];
    }
    
    x_impute ~ normal(x_missing_mean, x_missing_sd);
    
    sigma_beta_x ~ cauchy(0,1);
    nu_x ~ gamma(2,0.1);
    beta_x ~  student_t(nu_x,0,sigma_beta_x);
    alpha_x ~ normal(0,5);
    
    for(j in 1:N_var) {
        vector[N] x_col = x_merge[ , j];
        real beta = beta_x[j];
        real alpha = alpha_x[j];
        y ~ bernoulli_logit(alpha + beta * x_col);
    }
}

"

In [21]:
stage_2_regress_code_file = 'stage_2_regress.stan'
write_lines(x = stage_2_regress, path = stage_2_regress_code_file)
stage_2_regress_model = stan_model(file = stage_2_regress_code_file, verbose = FALSE)

DIAGNOSTIC(S) FROM PARSER:
Info:
Left-hand side of sampling statement (~) may contain a non-linear transform of a parameter or local variable.
If it does, you need to include a target += statement with the log absolute determinant of the Jacobian of the transform.
Left-hand-side of sampling statement:
    x_impute ~ normal(...)



In [22]:
###
# This function formats the data for
###

default_impute = function(x_censored, x_missing) {
    N_miss = dim(x_missing)[1]
    #print(N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

stage_2_regress_make_data = function(x_censored, y, IMPUTE_FUNC=default_impute, threshold=NULL, ...) {
    
    if(is.null(threshold)) {
        threshold = apply(x_censored,MARGIN=2,FUN=function(x) { min(x,na.rm = T)})
    }
    
    naive_impute_val = log( exp(threshold) / 2)
    
    n = dim(x_censored)[1]
    N_var = dim(x_censored)[2]
    N_miss = sum(is.na(x_censored))
    
    x_naive_impute = x_censored %>% select_all()
                 
    for(j in 1:N_var) {
        x_naive_impute[is.na(x_naive_impute[,j]),j] = naive_impute_val[j]
    }            


    x_censored[is.na(x_censored)] = missing_val

    x_censored = as.matrix(x_censored)
    x_missing = matrix(nrow = N_miss, ncol = 2)
    x_missing_init = rep(0,N_miss)
    x_missing_init_raw = rep(0,N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    # make a table were the first column is the row number in x_censored
    # second column contains the metabolite column number in x_censored
    col1 = c()
    col2 = c()
    for(j in 1:dim(x_censored)[2]) {
        missing_in_col = which(x_censored[,j] == missing_val)
        col1 = c(col1, missing_in_col)
        col2 = c(col2, rep(j,length(missing_in_col)))
    }
    x_missing[,1] = col1
    x_missing[,2] = col2
    
    # make a vector with naive imputation values
    if( N_miss > 0 ) {
        for(i in 1:dim(x_missing)[1]) {
            row = x_missing[i,1]
            col = x_missing[i,2]
            x_missing_init[i] = x_naive_impute[row,col]
            x_missing_init_raw[i] = x_missing_init[i] - threshold[col]
        }
    
        mean_sds = IMPUTE_FUNC(x_censored,x_missing,...)
        x_missing_mean = mean_sds$x_missing_mean
        x_missing_sd = mean_sds$x_missing_sd
    }
    
    data = list(
            N = n,
            N_miss = N_miss,
            N_var = N_var,
            x_raw = x_censored,
            x_naive_impute = as.matrix(x_naive_impute),
            x_missing = x_missing,
            x_missing_init = x_missing_init,
            x_missing_init_raw = x_missing_init_raw,
            x_missing_mean=x_missing_mean,
            x_missing_sd=x_missing_sd,
            threshold = threshold,
            y=y
        )

    #str(data)
    #mean(x_censored == missing_val)
    #data$N_miss / (data$N * data$N_var)

    data   
}


In [25]:
stage_1_impute = function(x_censored, x_missing, stage_1_post=NULL) {
    N_miss = dim(x_missing)[1]
    print(N_miss)
    x_missing_mean = apply( stage_1_post$x_impute_mean, FUN=mean, MARGIN=2 )
    x_missing_sd = apply( stage_1_post$x_impute, FUN=sd, MARGIN=2 )
            
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

fancy_impute = function(x_censored, x_missing) {
    N_miss = dim(x_missing)[1]
    #print(N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    
    x_obs = out$df_sample[,metabolites]
    for(i in 1:N_miss) {
        row = x_missing[i,1]
        col = x_missing[i,2]
        x_missing_mean[i] = x_obs[row,col]
        x_missing_sd[i] = 0.001
    }
            
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

stage_1_post = extract(stage_1_fit)
stage_2_data = stage_2_regress_make_data(x_censored, y, IMPUTE_FUNC = stage_1_impute, stage_1_post=stage_1_post)
#stage_2_init = list(
#    x_impute_raw = stage_2_data$x_missing_init_raw
#)


stage_2_init = replicate_init(
    list(
        x_impute_raw = stage_2_data$x_missing_init_raw
    ), chains=4
)


In [26]:
str(stage_2_data)
str(stage_2_init)

List of 12
 $ N                 : int 200
 $ N_miss            : int 0
 $ N_var             : int 27
 $ x_raw             : num [1:200, 1:27] -0.2539 0.1963 0.0995 1.0577 -0.5308 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_naive_impute    : num [1:200, 1:27] -0.2539 0.1963 0.0995 1.0577 -0.5308 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_missing         : int[0 , 1:2] 
 $ x_missing_init    : num(0) 
 $ x_missing_init_raw: num(0) 
 $ x_missing_mean    : num(0) 
 $ x_missing_sd      : num(0) 
 $ threshold         : Named num [1:27] -2.4 -2.11 -3.15 -2.4 -2.32 ...
  ..- attr(*, "names")= chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ y                 : num [1:200] 0 0 0 0 0 0 0 0 0 0 ...
List of 4
 $ 1:List of 1
  ..$ x_impute_raw: num(0) 


In [27]:
#stage_2_data$threshold
sum(apply(x_censored,FUN=function(x) { sum(is.na(x))},MARGIN=2))
x_censored[ stage_2_data$x_missing %>% head ]
out$df_sample[ stage_2_data$x_missing %>% head ]
stage_2_data$x_missing_mean[1:5]

[1] 0

numeric(0)

numeric(0)

[1] NA NA NA NA NA

In [28]:
stage_2_fit = sampling(stage_2_regress_model,  data = stage_2_data, init=stage_2_init, cores=4, chains=4, iter = 2000, control=list(adapt_delta=0.8))


In [30]:
df_sum = data.frame(summary(stage_2_fit)$summary) 
df_sum$PARAM = rownames(summary(stage_2_fit)$summary)
df_sum %>% arrange(n_eff)

mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
-3.693782e+03,0.139187971,5.43317507,-3.705451e+03,-3.697363e+03,-3.693335e+03,-3.689966e+03,-3684.0872810,1523.716,1.0004610,lp__
2.704567e-01,0.001151094,0.05846062,1.605048e-01,2.312148e-01,2.674315e-01,3.068455e-01,0.3943336,2579.319,1.0005271,sigma_beta_x
6.021443e-01,0.002339044,0.15654278,3.057414e-01,4.943876e-01,5.987772e-01,6.995075e-01,0.9251762,4479.084,0.9994188,beta_x[20]
1.888293e+01,0.206158283,13.82345025,2.644752e+00,8.770011e+00,1.545775e+01,2.553169e+01,54.0212228,4496.052,1.0004186,nu_x
2.421992e-01,0.001957589,0.13177027,-8.215799e-03,1.507370e-01,2.385330e-01,3.303737e-01,0.5151924,4530.979,1.0005709,beta_x[7]
1.905048e-01,0.001938952,0.13248659,-6.110603e-02,9.836836e-02,1.894657e-01,2.780878e-01,0.4546158,4668.848,0.9999990,beta_x[6]
5.356757e-01,0.002270873,0.15518584,2.413977e-01,4.278616e-01,5.299577e-01,6.391522e-01,0.8555636,4670.017,1.0001557,beta_x[5]
7.214471e-02,0.001803849,0.12580337,-1.759327e-01,-1.299364e-02,7.021999e-02,1.585221e-01,0.3155466,4863.892,0.9993466,beta_x[10]
1.590143e-03,0.002034303,0.14263441,-2.755646e-01,-9.599846e-02,1.958853e-03,9.791210e-02,0.2812297,4916.061,0.9993125,alpha_x[14]


### Run both models

In [32]:
make_summary_table_beta = function(fit,metabolites) {
    df_summary = data.frame(summary(fit)$summary)
    post = rstan::extract(fit)
    #compute prob > 0 or prob < 0
    n_sample = dim(post$beta_x)[1]
    g0 = colSums( (post$beta_x > 0) ) / n_sample 
    l0 = colSums( (post$beta_x < 0) ) / n_sample
    df_prob = data.frame(G0=g0,L0=l0) %>% rowwise() %>% mutate(Max=max(G0,L0))
    beta_names = c()
    for(i in seq(0,dim(post$beta_x)[2])) {
        beta_names=c(beta_names,paste0('beta_x[',i,']'))
    }
    df_summary_beta = df_summary %>% mutate(Param = rownames(.)) %>% 
    filter(Param %in% beta_names) %>% cbind(Metabolite = metabolites, P_GT_LT_0=df_prob$Max) %>%
    mutate(Z=mean/sd) %>% arrange(-P_GT_LT_0, -abs(Z)) %>% select(Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0)
    return(df_summary_beta)   
}

In [35]:
run_bayes_model = function(df_censored,metabolites,cores=4,chains=4,iter=2000, 
                           adapt_delta=0.8,
                           max_treedepth=10) {
    x_censored = df_censored[, metabolites]
    y = df_censored$died_90_day
    
    stage_1_data = stage_1_impute_make_data(x_censored, y, N_use_impute=8)
    stage_1_init_data = replicate_init(
        stage_1_impute_get_init_vals(stage_1_data), chains=4
    )
    
    control = list(adapt_delta=adapt_delta,max_treedepth=max_treedepth)
    print("CONTROL")
    print(control)
    
    N_miss = stage_1_data$N_miss
    stage_1_fit = NULL
    stage_1_post = NULL
    if(N_miss > 0) {
        print("IMPUTING")
        stage_1_fit = sampling(stage_1_impute_model,  data = stage_1_data, init=stage_1_init_data, cores=cores, chains=chains, iter = iter,
                             control = control )

        stage_1_post = extract(stage_1_fit)
    } else {
        print("NO MISING DATA SKIPPING IMPUTATION")
    }
    
    stage_2_data = stage_2_regress_make_data(x_censored, y, IMPUTE_FUNC = stage_1_impute, stage_1_post=stage_1_post)
    stage_2_init = list(
        x_impute_raw = stage_2_data$x_missing_init_raw
    )
    
    stage_2_init = replicate_init(
        list(
            x_impute_raw = stage_2_data$x_missing_init_raw
        ), chains=4
    )
    
    
    stage_2_fit = sampling(stage_2_regress_model,  data = stage_2_data, init=stage_2_init, cores=cores, chains=chains, iter = iter, control = control )

    
    sum_table = make_summary_table_beta(stage_2_fit,metabolites)
    rownames(sum_table) = sum_table$Metabolite
    list( sum_table=sum_table, stage_1_fit=stage_1_fit, stage_2_fit=stage_2_fit)
}

In [36]:
make_summary_table_beta(stage_2_fit,metabolites)

Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
oacetylcarnitine,beta_x[20],0.602144328,0.1565428,0.305741416,0.598777188,0.9251762,0.9994188,3.84651602,1.00000
propyleneglycol,beta_x[24],0.558118783,0.1549654,0.270281009,0.555420703,0.8730252,1.0008864,3.60157138,1.00000
betaine,beta_x[5],0.535675699,0.1551858,0.241397739,0.529957741,0.8555636,1.0001557,3.45183366,1.00000
glutamine,beta_x[12],0.482464553,0.1444542,0.204855062,0.478734391,0.7714754,1.0000340,3.33991378,1.00000
proline,beta_x[23],0.452164287,0.1431246,0.178446177,0.448447569,0.7513300,1.0015512,3.15923547,0.99950
alanine,beta_x[4],0.378291836,0.1385942,0.105486650,0.376257005,0.6581200,0.9997992,2.72949211,0.99875
choline,beta_x[7],0.242199188,0.1317703,-0.008215799,0.238532980,0.5151924,1.0005709,1.83804126,0.97125
phenylalanine,beta_x[22],0.231262937,0.1315713,-0.015673397,0.226820426,0.4987477,0.9995993,1.75769986,0.96825
pyruvate,beta_x[25],0.180844658,0.1254991,-0.060783435,0.178428434,0.4326891,0.9996341,1.44100391,0.92900


In [37]:
bayes_model_res = run_bayes_model(out$df_censored,metabolites,cores=4,chains=4,iter=2000, 
                           adapt_delta=0.8,
                           max_treedepth=10)

[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"


In [50]:
apply( out$df_censored, FUN=function(x) { mean(is.na(x)) }, MARGIN=1)

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [38] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[112] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[149] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[186] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [38]:
names(bayes_model_res)

[1] "sum_table"   "stage_1_fit" "stage_2_fit"

In [44]:
df_sum = data.frame(summary(bayes_model_res$stage_2_fit)$summary) 
df_sum$PARAM = rownames(summary(bayes_model_res$stage_2_fit)$summary)
df_sum %>% arrange(n_eff) %>% head(20)

,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,-3.693744e+03,0.119359238,5.25094572,-3.705023e+03,-3.697071e+03,-3.693466e+03,-3.690049e+03,-3684.3907669,1935.365,1.0008823,lp__
2,2.704614e-01,0.001201773,0.05922066,1.623408e-01,2.301179e-01,2.679717e-01,3.074278e-01,0.3934645,2428.297,1.0009080,sigma_beta_x
3,3.130437e-02,0.001900841,0.12424215,-2.186247e-01,-4.977224e-02,2.959231e-02,1.125665e-01,0.2697740,4272.150,0.9998907,beta_x[14]
4,1.877513e+01,0.209134657,13.80103725,2.547691e+00,8.355651e+00,1.557914e+01,2.533564e+01,53.1997754,4354.832,1.0021720,nu_x
5,7.992262e-02,0.001849886,0.12284955,-1.586489e-01,-3.339383e-03,7.969803e-02,1.596183e-01,0.3252992,4410.191,0.9993633,beta_x[8]
6,-3.121427e-04,0.002102411,0.13963214,-2.644853e-01,-9.375295e-02,-3.393980e-03,8.917245e-02,0.2778211,4410.986,1.0009781,alpha_x[2]
7,5.590588e-01,0.002310414,0.15380969,2.676331e-01,4.548068e-01,5.536385e-01,6.607943e-01,0.8705040,4431.876,0.9992926,beta_x[24]
8,2.381242e-01,0.002002448,0.13411411,-1.038857e-02,1.455772e-01,2.323347e-01,3.247966e-01,0.5114594,4485.660,1.0000196,beta_x[7]
9,1.592613e-01,0.001985475,0.13437922,-9.348927e-02,6.781162e-02,1.574157e-01,2.457116e-01,0.4358222,4580.738,1.0004235,beta_x[27]


In [45]:
bayes_model_res$sum_table

,Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
oacetylcarnitine,oacetylcarnitine,beta_x[20],0.602787124,0.1539285,0.31227112,0.599549253,0.9101505,0.9997817,3.91601984,1.00000
propyleneglycol,propyleneglycol,beta_x[24],0.559058761,0.1538097,0.26763308,0.553638509,0.8705040,0.9992926,3.63474343,1.00000
betaine,betaine,beta_x[5],0.533917729,0.1585924,0.23620866,0.530853538,0.8569967,0.9996059,3.36660441,1.00000
glutamine,glutamine,beta_x[12],0.487105314,0.1454465,0.20988755,0.482981210,0.7762367,0.9993455,3.34903332,0.99975
proline,proline,beta_x[23],0.449542052,0.1392841,0.18492483,0.446414833,0.7324860,0.9995653,3.22751770,0.99925
alanine,alanine,beta_x[4],0.376750856,0.1400757,0.11240965,0.373360049,0.6558879,1.0002947,2.68962312,0.99825
choline,choline,beta_x[7],0.238124203,0.1341141,-0.01038857,0.232334730,0.5114594,1.0000196,1.77553427,0.96975
phenylalanine,phenylalanine,beta_x[22],0.228276735,0.1324639,-0.03237749,0.228470741,0.4960043,0.9997902,1.72331270,0.96000
pyruvate,pyruvate,beta_x[25],0.178326327,0.1224995,-0.05463035,0.178700650,0.4277274,0.9998773,1.45573048,0.93025


### Test Full Simulation

In [46]:
run_sim = function(n_sim=100,n_0=100,n_1=100,frac_sig=0.4,
                   metabolites=NULL,censor=TRUE,max_missing=0.4,iter=2000,sim_number=1,MISSING_RATE=NULL,alpha=NULL,beta=NULL) {
    res = data.frame(stringsAsFactors = F)
    for(i in 1:n_sim) {
        sim_data = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=frac_sig,censor=censor,max_missing=max_missing,alpha = alpha, beta=beta)
        df_censored = sim_data$df_censored
        df_naive_impute = sim_data$df_naive_impute
        truth = sim_data$truth
        
        out_sim = paste0("RAW_", "SIM_NUMBER_",sim_number,"_N_SAMPLE_",n_sample,"_FRAC_SIG_",frac_sig,'_MISSING_RATE_',MISSING_RATE,'_SIM_DATA','.rds' )
        print("WRITNG OUT...")
        print(out_sim)
        write_rds(sim_data, out_sim, compress = 'gz')
        
        df_eff_sim = sapply(metabolites,FUN=function( x ) { apply_logistic_sim(x,df_naive_impute) } )
        sig_raw = as.vector( df_eff_sim['Pr(>|z|)',] < 0.05)
        eff_raw = as.vector( df_eff_sim['Estimate',])
        
        sig_bon = as.vector( p.adjust( df_eff_sim['Pr(>|z|)',],method='bonferroni') < 0.05)
        eff_bon = as.vector( df_eff_sim['Estimate',])
        
        sig_bh = as.vector( p.adjust( df_eff_sim['Pr(>|z|)',],method='BH') < 0.05)
        eff_bh = as.vector( df_eff_sim['Estimate',])
        
        bayes_model_res = run_bayes_model(df_censored,metabolites,cores=4,chains=4,iter=2000, 
                           adapt_delta=adapt_delta,
                           max_treedepth=max_treedepth)
        
        
        sum_table = bayes_model_res$sum_table
        stage_1_fit = bayes_model_res$stage_1_fit
        stage_2_fit = bayes_model_res$stage_2_fit
        
        out_fit = paste0("RAW_", "SIM_NUMBER_",sim_number,"_N_SAMPLE_",n_sample,"_FRAC_SIG_",frac_sig,'_MISSING_RATE_',MISSING_RATE,'_FIT_DATA','.rds' )
        print("WRITNG OUT...")
        print(out_fit)
        write_rds(bayes_model_res, out_fit, compress = 'gz')

        sig_bayes = as.vector( sum_table[metabolites,'P_GT_LT_0'] > 0.975  )
        eff_bayes = as.vector( sum_table[metabolites,'mean'] )
        
        res_bayes = compute_stats(sig_bayes,truth,'bayes', eff_bayes, true_effs )
        res = rbind(res,res_bayes,stringsAsFactors = F)
        
        res_raw = compute_stats(sig_raw,truth,'raw', eff_raw, true_effs )
        res = rbind(res,res_raw,stringsAsFactors = F)
        res_bon = compute_stats(sig_bon,truth,'bon', eff_bon, true_effs )
        res = rbind(res,res_bon,stringsAsFactors = F)
        res_bh = compute_stats(sig_bh,truth,'bh', eff_bh, true_effs )
        res = rbind(res,res_bh,stringsAsFactors = F)
        
    }
    res
}


In [47]:
apply_logistic_sim = function(met,df) {
    m = glm(died_90_day ~ I(df[,met]) ,data=df,family="binomial")
    summary(m)$coef[2,]
}

compute_stats = function(sig, truth, label, est_effs, true_effs ) {
    tp = sum(   (sig == 1) & (truth == 1)  )
    fn = sum(  (sig == 0) & (truth == 1) )
    fp = sum(  (sig == 1) & (truth == 0)   )
    tn =  sum(  (sig == 0) & (truth == 0)  )
    avg_mag_error = mean( abs(est_effs[sig & truth]/true_effs[sig & truth]) )
    num_sign_error = sum( (sign(est_effs) != sign(true_effs))[sig & truth] )
    n_sig = sum(sig)
    n_true = sum(truth)
    
    if(n_sig == 0) {
        avg_mag_error = 0
        num_sign_error = 0
    }
    return(list(label=label,tp=tp,fn=fn,fp=fp,tn=tn,avg_mag_error=avg_mag_error, num_sign_error=num_sign_error, n_sig=n_sig, n_true=n_true))
}


In [48]:
sim_data = make_df(10000,mu_0,sigma_0,10000,mu_1,sigma_1,frac_sig=1,censor=FALSE)
df_sample = sim_data$df_sample
truth = sim_data$truth
df_eff_sim = sapply(colnames(df_sample)[1:27],FUN=function( x ) { apply_logistic_sim(x,df_sample) } )
true_effs = as.numeric(df_eff_sim['Estimate',])
true_effs

[1]  0.163364533 -0.071909467  0.062469547  0.497537667  0.653125787
 [6]  0.375400572  0.228516668  0.296442698  0.248074240  0.397163651
[11]  0.007145001  0.544193714  0.560530087  0.349306688  0.055755496
[16]  0.528384923  0.195680221  0.539678610  0.491448915  0.742787623
[21]  0.260777675  0.352879483  0.484480118  0.613135152  0.257000717
[26]  0.656703858  0.170360191

In [52]:

n_sample = 100
frac_sig=0.4
max_missing=0.4
n_iter=2000
adapt_delta=0.8
max_treedepth=10
TARGET_MISSING_RATE = 0.05
beta_params = get_alpha_beta_for_target_rate(df_nmr,metabolites,TARGET_MISSING_RATE = TARGET_MISSING_RATE)
alpha = beta_params$alpha
beta = beta_params$beta
sim_number=1
res = run_sim(n_sim=1,n_0=n_sample,
              n_1=n_sample,frac_sig=frac_sig,metabolites=metabolites,
              max_missing=max_missing,iter=n_iter,sim_number=1,MISSING_RATE=TARGET_MISSING_RATE,
              alpha=alpha, beta=beta, censor=FALSE)


[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.4_MISSING_RATE_0.05_SIM_DATA.rds"
[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"
[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.4_MISSING_RATE_0.05_FIT_DATA.rds"
